# data logger for NIST group at 1-ID-E in 2021-06

**Objectives**

* Log a list of PVs that update asynchronous to sample measurements.
* List should be a text file that is easy to read and load.
* text file should not be too long so make a new file for each day

**Notes**

The [dhtioc](https://github.com/prjemian/dhtioc) project
has [datalogger](https://github.com/prjemian/dhtioc/blob/main/dhtioc/datalogger.py)
module that could be used as a starting point.

In [1]:
from ophyd import EpicsSignalRO
import datetime
import os
import threading
import time

In [2]:
class PvLogger:
    """
    Record raw values in data files.

    EXAMPLE::
    
        import pvlogger
        import time

        file_path = ""/tmp/pvlogger"
        pvlist = '''
            rpib4b1:temperature 
            rpib4b1:humidity 
            gp:UPTIME 
            gp:datetime
        '''.split()
        datalogger = pvlogger.PvLogger(pvlist, path=file_path)
        datalogger.start_recording()
        time.sleep(3600)  # log data for one hour at 10s (default) intervals
        datalogger.stop_recording()

    PARAMETERS

    pvlist
        *[str]* :
        list of EPICS PV name(s) to be logged
    path
        *str* :
        Base directory path under which to store data files.
        (default: ``~/Documents/pvlogger``)
    """

    def __init__(self, pvlist, path=None):
        """Constructor."""
        print(f"PvLogger starting")
        self.pvs = {
            f"pv{i+1}": EpicsSignalRO(pv, name=f"{pv}{i+1}")
            for i, pv in enumerate(pvlist)
        }
        self.base_path = path or os.path.abspath(
            os.path.join(
                os.environ.get("HOME", os.path.join("/", "home", "pi")),
                "Documents",
                "pvlogger",
            )
        )
        self.file_extension = "txt"
        self.recording = None
        self.recording_period = None
        self._request_stop_recording = False
        self.recording_poll_delay = 0.1  # seconds

    def get_daily_file(self, when=None):
        """
        Return absolute path to daily file.

        PARAMETERS

        when
            *obj* :
            Path will be based on this instance of `datetime.datetime`.
            (default: now)
        """
        dt = when or datetime.datetime.now()
        path = os.path.join(
            self.base_path,
            f"{dt.year:04d}",
            f"{dt.month:02d}",
            (
                f"{dt.year:04d}"
                f"-{dt.month:02d}"
                f"-{dt.day:02d}"
                f".{self.file_extension}"
            ),
        )
        return path

    def create_file(self, fname):
        """
        Create the data file (and path as necessary)

        PARAMETERS

        fname
            *str* :
            File to be created.  Absolute path.
        """
        path = os.path.split(fname)[0]

        # create path as needed
        os.makedirs(path, exist_ok=True)
        if not os.path.exists(path):
            raise FileNotFoundError(
                f"Could not create directory path: {path}"
            )

        # create file
        with open(fname, "w") as f:
            created = datetime.datetime.now().isoformat(sep=" ")
            header = (
                f"# file: {fname}\n"
                f"# created: {created}\n"
                "# program: pvlogger\n"
                "# column separator: tab (^T or \\t)\n"
                "#\n"
                "# time: (UTC) seconds (since 1970-01-01T00:00:00 UTC)\n"
            )
            # for key, pv in self.pvs.items():
            #     header += f"# {key}: {pv.pvname}\n"
            header += "# \n"
            header += "# time\t"
            header += "\t".join([pv.pvname for pv in self.pvs.values()])
            header += "\tymd hms\n"
            f.write(header)

    def record(self, when=None):
        """
        Record new PV values.  Create new file and path as needed.

        PARAMETERS

        when
            *obj* :
            `datetime.datetime` of these values.
            (default: now)
        """
        dt = when or datetime.datetime.now()
        fname = self.get_daily_file(dt)
        try:
            if not os.path.exists(fname):
                self.create_file(fname)
            with open(fname, "a") as f:
                record = [f"{dt.timestamp():.02f}",]
                for pv in self.pvs.values():
                    record.append(f"{pv.get()}")
                record.append(f"{dt}")
                f.write("\t".join(record)+"\n")
                print("\t".join(record))
        except Exception as exc:
            print(f"Continuing after exception: {exc}")

    def start_recording(self, period=10):
        """
        Initiate periodic recording (or change period).

        PARAMETERS

        period
            *number* :
            interval (seconds) between recorded measurements.
            Minimum interval is 0.5 seconds.  Default is 10 seconds.
        """
        if period is None:
            period = 10
        period = max(period, 0.5)
        if self.recording_period != period:
            print(
                f"changing recording period from {self.recording_period}s"
                f" to {period}s"
            )
            self.recording_period = period

        if self.recording is not None:
            print("Already recording...  Will not restart.")
            return

        def worker():
            """Background thread that orders recording."""
            print("Periodic recording thread starting...")
            # wait for all PVs to connect
            for pv in self.pvs.values():
                pv.wait_for_connection()
            next_recording = time.time()
            while not self._request_stop_recording:
                if time.time() >= next_recording:
                    next_recording += self.recording_period
                    self.record()
                time.sleep(self.recording_poll_delay)
            self._request_stop_recording = False
            self.recording = None
            print("Periodic recording thread exiting...")

        self._request_stop_recording = False
        self.recording = threading.Thread(target=worker, daemon=True)
        self.recording.start()

    def stop_recording(self):
        if self.recording is not None:
            self._request_stop_recording = True

In [3]:
lager = PvLogger("rpib4b1:temperature rpib4b1:humidity gp:UPTIME gp:datetime".split(), path="/tmp")

PvLogger starting


In [4]:
lager.start_recording()

changing recording period from Nones to 10s
Periodic recording thread starting...


In [5]:
time.sleep(10*60)

1624658074.35	25.700009936760097	99.9	19 days, 01:29:04	2021-06-25 21:54:34	2021-06-25 16:54:34.345218
1624658084.41	25.700001922681235	99.9	19 days, 01:29:14	2021-06-25 21:54:44	2021-06-25 16:54:44.407528
1624658094.41	25.700000372022988	99.9	19 days, 01:29:24	2021-06-25 21:54:54	2021-06-25 16:54:54.412975
1624658104.35	25.700000071983386	99.9	19 days, 01:29:34	2021-06-25 21:55:04	2021-06-25 16:55:04.349858
1624658114.42	25.700000013928193	99.9	19 days, 01:29:44	2021-06-25 21:55:14	2021-06-25 16:55:14.423728
1624658124.42	25.700000002694996	99.9	19 days, 01:29:54	2021-06-25 21:55:24	2021-06-25 16:55:24.418529
1624658134.35	25.700000000521463	99.9	19 days, 01:30:04	2021-06-25 21:55:34	2021-06-25 16:55:34.347838
1624658144.42	25.7000000001009	99.9	19 days, 01:30:14	2021-06-25 21:55:44	2021-06-25 16:55:44.424704
1624658154.41	25.70000000001953	99.9	19 days, 01:30:24	2021-06-25 21:55:54	2021-06-25 16:55:54.410749
1624658164.36	25.70000000000378	99.9	19 days, 01:30:34	2021-06-25 21:56:04	2

In [6]:
lager.stop_recording()

Periodic recording thread exiting...
